# Demonstration Notebook

This notebook is meant **only for final demonstration of output** of Machine Learning training applied over CFD data collected. All necessary functions and transformation is carried under the hood in background files.

** jupyter-nbextension enable toc/toc**

**Things to add:**

- Model evolution
- Video files of training data and prediction
- Prediction on a new data
- Content
- Eta study
- Pressure, Lift, Drag calculation
- Before PPT: Simulation assumptions methods and general airfoil info
- Before PPT: CFD Files and Simulation details
- Before PPT: Novel features of this approach
- Before PPT: Reliability and Usage
- Before PPT: Significance and future work

### CFD-ML
Combining domains of **Computational Fluid Dynamics** and **Machine Learning**. Our aim is to help the community of chaos modelling and CFD in reducing the need for high-end computations and accurate and bulky data collection during experiments.

<p align="center">
    <img src="random_images/airfoil-1.jpg" width="500" align="left" style= "margin: 1px 2px">
    <img src="random_images/dl.png" width="400" align="right" style= "margin: 1px 2px">
</p>

## Methodology
<img src="random_images/idea.png" width="500" align="left" style="margin:5px 10px"/>

We seek to create continuous fluid flow predictions which align well with corresponding CFD simulations. Our idea is to gather data scattered in space and time and try to learn the transformation function using **deep neural networks**.

This function takes the co-ordinates of any point in space-time co-ordinate system (x, y, t) as inputs and predicts the flow field values (u, v, p) as output of it. An ideal transformation function would be able to stay close to simulation data while satisfying **Navier-Stokes equation**. This imposes another level on the current challenge to derive this function. 

Once we have the function, we simply need to pass the all the co-ordinate values of a given time-snap to get the **simulation snapshot at that time step**.


## Data and Model
Make sure model is the one with least mean squared error and corresponding to data file chosen

In [1]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
%matplotlib inline
%matplotlib notebook
import essential_functions as ef
from essential_functions import *
import os

/home/ritvik/anaconda3/envs/cfd_ml/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ritvik/anaconda3/envs/cfd_ml/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ritvik/anaconda3/envs/cfd_ml/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ritvik/anaconda3/envs/cfd_ml/lib/python3

In [2]:
_MODEL_DIR = "kaggle_models/"
_DATA_DIR = "data/"

_DATA = "data/final_translated_data_1_3.csv"
_MODEL = "trained_models/medium_foil_model4.h5"#"kaggle_models/model_3_0-0125.h5"

## Load, convert and scale data

Data is modified to have a perspective with respect to **centre of mass** of airfoil. All further calculations and plots would be based on the same.
The displacement of airfoil in experimental setup is dealt independently in the solution.

In [3]:
data = load_data(filename=_DATA)
t_orig,x_orig,y_orig,u_orig,v_orig,p_orig,eta_orig = create_tensors(data)
scaler = scale_learn(t_orig,x_orig,y_orig,u_orig,v_orig,p_orig,eta_orig)
t,x,y,u,v,p,eta = scale(scaler,t_orig,x_orig,y_orig,u_orig,v_orig,p_orig,eta_orig)
print("[INFO] Number of data points: " +str(t.shape[0]))

[INFO] Time taken = 5.7555084228515625 seconds.
[INFO] Number of data points: 3886682


In [4]:
# sub_data = data.sample(frac=0.1)
# fig=plot3D(sub_data["x"],sub_data["t"],sub_data["y"],s=0.1,saved=True)
# plot_time_data(2.00,data)

## Data Plots
As it can be seen, the data collected is neither continuous in co-ordinate scales nor in time steps. The total number of data-points we have are over **3.8 million** spread out evenly between **300 time slices. (0.01sec/slice)**.

### Data transformation
Data transformation had the following steps: 
1. **Scraping** useful data from individual simulation files for individual time-steps. Also filtering the domain data from the even larger simulation domain.
2. **Flattening** to 2D and filtering values which are used for model training and evaluation. Force and Torque acting is also preserved for future use.
3. **Performing selectively random data selection** for capturing different percentage of different type of points (explained below). This required finding the exact co-ordinates of surface points at each given instant of time.
4. **Uniform time-selection** of data to uniformly select same number of data points from each time frame.
5. **Shift of reference frame** from experiment reference from to centre of airfoil reference frame. This is required to simplify the complexity of problem where the model has to **deal with sudden absence of fluid** in a region when airfoil is oscillating up and down. In centre-of-mass of airfoil frame, airfoil remains stationary and hence no such sudden absence. We are also rearning the mapping of y-directional oscillation of the airfoil so as to be able to revert back to our original settings after prediction.

### Data selection
The co-ordinate selection is selectively random according to following rule:
1. **Points attached to body** are given highest priority (~70% points from the mesh got selected). This needed to capture boundary layer equation.
2. **Points in the viscinity** of previous layer also face the similar problem and hence are given second highest probability
3. **Any other points** in the chosen domain has a probability of selection ~30%. Domains is 2D and extends from **(-c,-1.2c) to (3c,1.2c)**.


<p float="left">
    <img align=left src="Figures/plot3d_show.png" alt="space-time data plot" width="400">
    <img align=left src="Figures/data_2_sec.png" alt="data-points at time: 3.5 sec" width="500">
</p>

## Comparison with DeepVIV
DeepVIV (Vortex Induced Vibrations) was the paper which led us to this approach. Their setup was on a much simpler and symmetric problem of flow over an oscillating cylinder while ours uses a more complex asmmetric oscillating airfoil. It would not be suitable to continue with their learning parameters and to compensate for the same, we have changed a few points of focus thereby keeping the general idea and architecture same.

|Property|DeepVIV|DeepFoil|
|-------|-------|--------|
|**Time-Snap of data**|![](random_images/deepviv_datasnap.png)|![](Figures/data_2_sec.png)|
|**Boundary condition**|Symmetric|Asymmetric|
|**Total data points**|~ 4 million|~ 3.8 million **(Less data)**|
|**Number of time frames**|280|200|
|**Inlet Reynold's number**|1000 (diameter based)|1000 (chord based)|
|**Number of oscillations completed**|2 in 14sec|3 in 2sec **(More turbulent)**|
|**Simulation Time**|14 sec|2 sec **(Much faster oscillations)**|
|**Model Architechture**|Deep fully connected sequential neural network|Deep fully connected sequential neural network|
|**Hidden layers**|10 layers|12 layers|
|**Width of model**|Uniform: 32/output|Uniform: 32/output|
|**Final Error**|around 2%|around 6%|

## Load and compile model

In [5]:
def custom_loss_wrapper(Re):
#     input_tensor = concatenate([t, x, y], 1)
    def gradient_calc(Re):
        
        uvp = model.output
        u = uvp[:,0:1]
        v = uvp[:,1:2]
        p = uvp[:,2:3]
        eta = uvp[:,3:4]
#         print(u)
        
        u_t,u_x,u_y = K.gradients(u,model.input)
        v_t,v_x,v_y = K.gradients(v,model.input)
        p_t,p_x,p_y = K.gradients(p,model.input)
        eta_t,eta_x,eta_y = K.gradients(eta,model.input)
        
        u_xx = K.gradients(u_x,model.input[1])[0]
        u_yy = K.gradients(u_y,model.input[2])[0]
        v_xx = K.gradients(v_x,model.input[1])[0]
        v_yy = K.gradients(v_y,model.input[2])[0]
        eta_tt = K.gradients(eta_t,model.input[0])[0]
        
#         print((u_xx)+(u_yy))
        
        eq1 = u_t + (u*u_x + v*u_y) + p_x - (1.0/Re)*(u_xx + u_yy)
        eq2 = v_t + (u*v_x + v*v_y) + p_y - (1.0/Re)*(v_xx + v_yy) + eta_tt
        eq3 = u_x + v_y
        
        loss = K.mean(tf.square(eq1)) + K.mean(tf.square(eq2)) + K.mean(tf.square(eq3))
        
#         print((u_xx))
        return loss

    def custom_loss(y_true, y_pred):
        navier_loss = gradient_calc(Re=Re)
#         navier_loss = net_VIV(input_tensor,y_pred,Re=1000)
        return tf.reduce_mean(tf.square(y_true - y_pred)) + navier_loss
    return custom_loss


## Loss function = Mean Squared Error (MSE) + Navier_loss

### Navier loss
<img src="random_images/navier-loss.png" align="left" width="400"/>

In [6]:
model = load_model(_MODEL,compile=False)
model.compile(loss=custom_loss_wrapper(Re=1000), optimizer='adam', metrics=['mse'])
# model.summary()

## Model Architechture

<p>Branched model consisting of 2 branches. One for flow prediction (u,v,p) and other for displacement of body (eta). Flow prediction is relatively more complex problem and hence requires wider network.</p>

**1. uvp_predictor**
<ul>
    <li>Inputs: x,y,t</li>
    <li>Outputs: u,v,p</li>
    <li>Layer count: 15</li>
    <li>Layer width: 32*3</li>
    <li>Layer type: Dense, BatchNormalization</li>
</ul>

**2. eta_predictor**
<ul>
    <li>Inputs: t</li>
    <li>Outputs: eta</li>
    <li>Layer count: 15</li>
    <li>Layer width: 32</li>
    <li>Layer type: Dense, BatchNormalization</li>
</ul>

### Loss function
<p>Custom Loss: RMSE loss (0.004) + Navier loss (0.0002)<br />
<strong>Navier Loss</strong>: error in the navier function at any given point. In final model, it gives rise to only 0.2% error.</p>

<img src="Figures/architecture.png"/>

## Results

**Flow at time : 2.3 sec**

<img src="Figures/flow_23sec.png" style="margin: 0px 2px" align="left" width="700px">

**RMSE errors**:
<p>u : 3.23%<br />v : 2.38%<br />p : 1.71%</p>

**Summary:** The model is able to capture flow trend. Most of the error is caused due to mismatch in eta prediction which can be seen in error in v, which has highest dependence on eta. p had the highest error in deepVIV too and so in our case although the methods of obtaining p are different. The reason for this anomaly is to be discussed. The error in direct pressure prediction (our approach) is relatively less.

**How to correct:** Improving eta prediction must be able to address this problem to some extent. Even after that, A bigger model may capture the intricasies to more detail.

**Is it enough:** Yes and No. If our aim is to substitute the current methods, we are far off. But this project had the focus of proving the capabilities of neural network to be able to learn more than that in deepVIV specially when the conditions are asymmetrical.

## Eta study and plot with time

<img src="Figures/eta_3plot.png" style="margin: 0px 2px" align="left" width="400">

**Summary:** The model is able to capture eta trend but is failing slightly to match due to such high change with respect to time. Most of the error in flow prediction is caused due to wrong eta provided to navier-loss.

**How to correct:** more time frames required. Currently working on the same. Would require even larger model to encompass this doubling of information.

## Add Lift-Drag calculation Here
Both training data and prediction

## Video

Shows training, prediction and error evolution with time 

<video src="Figures/flow.mp4" controls>

## Ask sir
1. Haven't applied p calculation from u and v
2. Haven't computed lift and drag from deepVIV formulae

## Graphical UI for demonstration

In [7]:
from ipywidgets import *
def predict_and_plot(time_val):
    u_snap,v_snap,p_snap,eta_snap=time_snap(time_val,scaler,data,model,plot=True,print_error=True)
    print(time_val)

In [11]:
time_slider = widgets.FloatSlider(value=2.3,min=1,max=3.0,step=0.01,description='Time',layout=Layout(width='50%', height='50px'))
time_slider.style.handle_color = '#0b6b66'
v=interact_manual(predict_and_plot, time_val= time_slider,description="Predict for time snap")

interactive(children=(FloatSlider(value=2.3, description='Time', layout=Layout(height='50px', width='50%'), ma…